In [1]:
# Colab Cell: Full pipeline with KD-string fix + numeric→strength mapping + correct Luke handling
#             + ADDITIVE S11 rescue + lookup
# Key points:
# - Keep legacy Source 11 (11). Never delete/remap (0,11, ...).
# - Fuzzy 0..4 strength parsing (strings/floats) AND 0..4 from numeric Kd strings (± handled; default unit = uM).
# - Never confuse presence (0,key) with Luke numeric (delta,n).
# - Prefer Luke's key as source only when *only* Luke numeric is present (no KD/strength).
# - S11 Rescue: *additively* re-inject dropped (0,11,strength) + Luke numeric from a baseline workbook.
# - Heatmap guard: never auto-fill kd_idx_auto for heatmap rows.
# - One-time cleanup of blank/'nan' rows without data + targeted placeholder rows without payload.
# - Interactive lookup.
#
# ADDITIONS IN THIS VERSION:
# - NEW sheet "Luke's paper plus hits" that applies delta>=0.15 as "binding":
#     * delta < 0.15 or blank => Bucket 0
#     * delta >= 0.15 => quartiles into Buckets 1-4 based ONLY on those >= 0.15
# - Propagate those new bucket strengths into Kd_matrix_indexed tuples (pos3) WHEN pos3 is NA/blank,
#   so Luke-derived interactions carry their strength into the matrix without overwriting existing pos3.
# - Ensure Mod Indexes updates when new mod tags appear in residues: discover missing tags and extend MOD_INDEX.
#
# ADDITIONS IN THIS VERSION (RE-ADD):
# - Re-add "Sequences" sheet that lists general histones + all reader names with a blank Sequence column.
#   If an existing "Sequences" sheet exists, preserve existing sequences and only add missing entries.

import re, ast, pandas as pd, numpy as np
from pathlib import Path
from google.colab import files

# ---------- 1) Upload workbook ----------
print("🔄 Upload your latest kd_master .xlsx:")
uploaded = files.upload()
kd_file = Path(next(iter(uploaded)))
xls     = pd.ExcelFile(kd_file)

# ---------- 2) Load core sheets ----------
orig_df       = pd.read_excel(xls, "Kd_matrix_indexed", dtype=str)
refs_df       = pd.read_excel(xls, "References")
keys_existing = pd.read_excel(xls, "Keys")

keys_existing.columns = (
    keys_existing.columns.str.strip()
        .str.lower()
        .str.replace(" ", "_")
)

existing_meta = {}
for _, r in keys_existing.iterrows():
    for fn in re.split(r"[;,]\s*", str(r.filename)):
        fn = fn.strip()
        if not fn: continue
        existing_meta[fn] = {
            "source_name":        r.source_name,
            "URL":                r.url,
            "measurement_method": r.measurement_method
        }

# ---------- 3) Overlay loader ----------
print("\nEnter NEW overlay files (CSV/XLSX); blank to stop.")
print(" Already have:", ", ".join(existing_meta.keys()) or "none")
overlay_paths = []
while True:
    p = input("Overlay file: ").strip()
    if not p: break
    if p in existing_meta:
        print(" → Skipping already-loaded:", p)
    else:
        overlay_paths.append(Path(p))

def _norm(df):
    m = {}
    for c in df.columns:
        lc = c.lower()
        if "reader"  in lc:   m[c] = "Histone readers"
        elif "residue" in lc: m[c] = "Histone residues"
        elif "kd"      in lc: m[c] = "Kd"
        elif "ref"     in lc: m[c] = "Reference"
        else:                 m[c] = c
    return df.rename(columns=m).rename_axis(None, axis=1)

for ov in overlay_paths:
    df0 = pd.read_csv(ov) if ov.suffix.lower()==".csv" else pd.read_excel(ov)
    df1 = _norm(df0)
    rows = [{
        "Histone readers":  str(r.get("Histone readers","")).strip(),
        "Histone residues": str(r.get("Histone residues","")).strip(),
        "Kd":               str(r.get("Kd","")).strip(),
        "Reference":        (str(r.get("Reference","")).strip() or ov.name)
    } for _, r in df1.iterrows()]
    refs_df = pd.concat([refs_df, pd.DataFrame(rows)], ignore_index=True)

# ---------- 5) Manual Kd entry ----------
print("\nManual Kd entry – blank reader to finish.")
while True:
    rdr = input("Reader: ").strip()
    if not rdr: break
    res = input("Residue: ").strip()
    kd  = input("Kd: ").strip()
    rf  = input("Reference/URL/DOI: ").strip() or "manual"
    refs_df = pd.concat([refs_df, pd.DataFrame([{
        "Histone readers":  rdr,
        "Histone residues": res,
        "Kd":               kd,
        "Reference":        rf
    }])], ignore_index=True)

# ---------- 6) Clean refs ----------
refs_df["Histone readers"]  = refs_df["Histone readers"].astype(str).str.replace(r"\(.*\)","",regex=True).str.strip()
refs_df["Histone residues"] = refs_df["Histone residues"].astype(str).str.strip()
refs_df["Kd"]               = refs_df["Kd"].astype(str).str.strip()
refs_df["Reference"]        = refs_df["Reference"].astype(str).str.strip()

bad_mask = refs_df["Histone residues"].str.len().eq(0) | refs_df["Histone residues"].isin({"#", "nan", "NaN", "None"})
if bad_mask.any():
    print(f"ℹ️  Skipping {bad_mask.sum()} rows with invalid residue labels (#/empty/nan).")
refs_df = refs_df[~bad_mask].drop_duplicates(["Histone readers","Histone residues","Kd","Reference"]).reset_index(drop=True)

# --- Heatmap guard: never auto-fill kd_idx_auto for heatmap rows
HEATMAP_REF = "heatmap_matrix_output_cleaned_corrected.xlsx"
if "kd_idx_auto" not in refs_df.columns:
    refs_df["kd_idx_auto"] = np.nan
mask_heatmap = refs_df["Reference"].astype(str).str.strip().eq(HEATMAP_REF)
refs_df.loc[mask_heatmap, "kd_idx_auto"] = np.nan

# ---------- 7) Build/refresh Keys ----------
raw_refs = list(refs_df["Reference"].unique())
all_keys = []
for ref in raw_refs:
    if ref in existing_meta:
        m = existing_meta[ref]
        all_keys.append({
            "filename":           ref,
            "source_name":        m["source_name"],
            "URL":                m["URL"],
            "measurement_method": m["measurement_method"]
        })
    else:
        src = input(f"Source name for {ref}: ").strip() or ref
        url = input(f"URL for {ref}: ").strip()
        mm  = input(f"Measurement for {ref}: ").strip()
        all_keys.append({
            "filename":           ref,
            "source_name":        src,
            "URL":                url,
            "measurement_method": mm
        })

grouped = {}
for k in all_keys:
    key = str(k["source_name"]).strip().lower()
    grp = grouped.setdefault(key, {
        "source_name": k["source_name"],
        "filenames":   [], "urls": [], "methods": []
    })
    grp["filenames"].append(k["filename"])
    grp["urls"].append(k["URL"])
    grp["methods"].append(k["measurement_method"])

entries = []
for i, grp in enumerate(grouped.values(), start=1):
    entries.append({
        "key":                i,
        "filename":           " , ".join(dict.fromkeys(map(str, grp["filenames"]))),
        "source_name":        grp["source_name"],
        "URL":                " , ".join(map(str, dict.fromkeys(grp["urls"]))),
        "measurement_method": " , ".join(map(str, dict.fromkeys(grp["methods"])) )
    })
keys_df = pd.DataFrame(entries)

ref_to_key = {}
for _, row in keys_df.iterrows():
    for fn in str(row["filename"]).split(" , "):
        fn = fn.strip()
        if fn:
            ref_to_key[fn] = row["key"]

# ---------- 8) Read Kd matrix and parse tuple strings ----------
raw_mat = pd.read_excel(xls, "Kd_matrix_indexed", dtype=str)
matrix = raw_mat.set_index("Histone residues") if "Histone residues" in raw_mat.columns else raw_mat
matrix.index = matrix.index.map(lambda x: "" if pd.isna(x) else str(x).strip())
matrix = matrix.loc[:, [c for c in matrix.columns if not str(c).lower().startswith("unnamed")]]

def parse_tuple_cell_to_list(cell):
    if cell in (None, np.nan): return []
    s = str(cell).strip()
    if not s or s == "(0,)": return []
    out = []
    for part in [p.strip() for p in s.split(";") if p.strip()]:
        if not (part.startswith("(") and part.endswith(")")):
            continue
        try:
            tup = ast.literal_eval(part)
            if isinstance(tup, tuple):
                out.append(tup)
        except Exception:
            pass
    return out

def list_to_tuple_cell(lst):
    if not lst: return "(0,)"
    return "; ".join("(" + ", ".join(repr(x) for x in t) + ")" for t in lst)

matrix_parsed = matrix.applymap(parse_tuple_cell_to_list)

# NEW: helper used in 8b and 11
def _valid_residue_label(x):
    if pd.isna(x):
        return False
    s = str(x).strip()
    return s != "" and s.lower() != "nan"

# ---------- 8b) S11 RESCUE (ADDITIVE) ----------
from google.colab import files as _files_rescue

def _is_presence_pair(t):
    return (isinstance(t, tuple) and len(t)==2 and t[0]==0
            and isinstance(t[1], (int,float)) and float(t[1]).is_integer())

def _strength_from_any(x):
    _WORDS = {
        "no spot":0,"nospot":0,"none":0,"0":0,"0.0":0,
        "very weak":1,"weak":1,"1":1,"1.0":1,
        "moderate":2,"mod":2,"2":2,"2.0":2,
        "strong":3,"3":3,"3.0":3,
        "very strong":4,"vstrong":4,"4":4,"4.0":4,
    }
    if isinstance(x, (int, float)):
        v = int(round(float(x)))
        return v if 0 <= v <= 4 else None
    s = str(x).strip().lower()
    if s in _WORDS: return _WORDS[s]
    m = re.search(r'\b([0-4])(?:\.0+)?\b', s.replace(",", " "))
    return int(m.group(1)) if m else None

def _find_luke_pair(items):
    for t in items if isinstance(items, list) else []:
        if isinstance(t, tuple) and len(t)==2 and all(isinstance(x,(int,float)) for x in t):
            if _is_presence_pair(t):  # skip (0, key)
                continue
            v, n = t
            return (float(v), int(n))
    return None

def _has_s11_triple(items, s11_key=11):
    if not isinstance(items, list): return False
    for t in items:
        if isinstance(t, tuple) and len(t)>=3:
            try:
                if t[0]==0 and int(t[1])==int(s11_key):
                    return True
            except Exception:
                pass
    return False

print("\n(Optional) Upload a BASELINE workbook (has the original (0, 11, strength) triples) to rescue Source 11:")
_rescue = _files_rescue.upload()
if _rescue:
    rescue_path = Path(next(iter(_rescue)))
    rescue_xls  = pd.ExcelFile(rescue_path)
    rescue_df   = pd.read_excel(rescue_xls, "Kd_matrix_indexed", dtype=str)
    rescue_df   = rescue_df.set_index("Histone residues")
    rescue_df   = rescue_df.loc[:, [c for c in rescue_df.columns if not str(c).lower().startswith("unnamed")]]
    rescue_parsed = rescue_df.applymap(parse_tuple_cell_to_list)

    s11_key = 11
    added_s11_to_empty = added_s11_to_nonempty = added_luke_pairs = 0

    # Ensure all readers/cols exist
    for rdr in rescue_parsed.columns:
        if rdr not in matrix_parsed.columns:
            matrix_parsed[rdr] = [list() for _ in range(len(matrix_parsed.index))]

    # Only add rows with a real, non-empty label
    need_rows = [r for r in rescue_parsed.index
                 if _valid_residue_label(r) and r not in matrix_parsed.index]
    if need_rows:
        add = pd.DataFrame({c: [list() for _ in range(len(need_rows))] for c in matrix_parsed.columns},
                           index=need_rows)
        matrix_parsed = pd.concat([matrix_parsed, add], axis=0)

    # Additive merge of S11 triples and Luke numeric pairs
    for res in rescue_parsed.index:
        for rdr in rescue_parsed.columns:
            old_items = rescue_parsed.at[res, rdr]
            if not isinstance(old_items, list) or len(old_items)==0:
                continue

            # strongest S11 strength in baseline
            s11_strength = None
            for t in old_items:
                if isinstance(t, tuple) and len(t)>=3 and t[0]==0:
                    try: key = int(t[1])
                    except Exception: continue
                    if key == s11_key:
                        st = _strength_from_any(t[2])
                        if st is not None and (s11_strength is None or st > s11_strength):
                            s11_strength = st

            luke_pair = _find_luke_pair(old_items)

            now_items = matrix_parsed.at[res, rdr]
            if not isinstance(now_items, list):
                now_items = []
            had_any = len(now_items) > 0
            had_s11 = _has_s11_triple(now_items, s11_key=s11_key)

            if s11_strength is not None and not had_s11:
                now_items.append((0, s11_key, s11_strength))
                if had_any: added_s11_to_nonempty += 1
                else:       added_s11_to_empty    += 1

            if luke_pair is not None:
                already_have_pair = any(
                    (isinstance(t, tuple) and len(t)==2 and not _is_presence_pair(t) and
                     abs(float(t[0]) - float(luke_pair[0])) < 1e-9 and int(t[1]) == int(luke_pair[1]))
                    for t in now_items
                )
                if not already_have_pair:
                    now_items.append((round(float(luke_pair[0]), 4), int(luke_pair[1])))
                    added_luke_pairs += 1

            matrix_parsed.at[res, rdr] = now_items

    print(f"✅ S11 rescue complete. Added S11 to empty cells: {added_s11_to_empty}; "
          f"to non-empty cells: {added_s11_to_nonempty}; Luke pairs added: {added_luke_pairs}")
else:
    print("⏭️ Skipping S11 rescue (no baseline uploaded).")

# ---------- 9) Residue canonicalization ----------
def canonicalize_residue_for_matrix(raw_res, idx):
    r = str(raw_res).strip()
    if not r or r.lower() in {"nan", "#"}:
        return None
    r = re.sub(r"\s*\(.*\)", "", r)
    r = re.sub(r"\s+", "", r)
    if r in idx:
        return r
    m = re.match(r'^(H2)K(\d+)([A-Za-z0-9]+)$', r, flags=re.I)
    if m:
        site, mod = m.group(2), m.group(3)
        candidates = [f"H2AK{site}{mod}", f"H2BK{site}{mod}"]
        available  = [c for c in candidates if any(str(x).startswith(c) for x in idx)]
        if len(available) == 1:
            return available[0]
        if len(available) > 1:
            print(f"❓ Ambiguous residue '{r}'. Options:")
            for i, c in enumerate(available, 1): print(f"   {i}) {c}")
            sel = input("   Choose 1/2 (blank to skip): ").strip()
            if sel.isdigit() and 1 <= int(sel) <= len(available):
                return available[int(sel)-1]
            return None
    matches = [x for x in idx if str(x).startswith(r)]
    if len(matches) == 1:
        return matches[0]
    return None

# ---------- S11 key (keep legacy 11 intact) ----------
S11_NAME = "Histone Recognition and Large-Scale Structural Analysis of the Human Bromodomain Family"
s11_key = 11  # do NOT remap legacy 11

# ---------- 10) Merge overlay refs into matrix_parsed ----------
DENYLIST_NON_HISTONE = {
    "dnamotif","h1","h1.4","h2ax","h2aac","h2b","h2bac","h3","h3andh4",
    "h3r2","h3ac","h4","h4kac","h4ac","hkme","mcg"
}

for _, row in refs_df.iterrows():
    rdr     = row["Histone readers"]
    raw_res = str(row["Histone residues"])
    raw_res_norm = re.sub(r"\s+", "", raw_res).strip().lower()

    if raw_res_norm in DENYLIST_NON_HISTONE:
        print(f"⏭️ Skipping denylisted category-like residue '{raw_res}'.")
        continue

    canon   = canonicalize_residue_for_matrix(raw_res, matrix_parsed.index)
    if not canon:
        print(f"⚠️  Residue '{raw_res}' not found/ambiguous; skipping.")
        continue
    if rdr not in matrix_parsed.columns:
        matrix_parsed[rdr] = [list() for _ in range(len(matrix_parsed.index))]
    ref_name = row["Reference"]
    if ref_name not in ref_to_key:
        print(f"⚠️  Reference '{ref_name}' has no key; skipping this entry.")
        continue
    k = ref_to_key[ref_name]
    raw = str(row["Kd"]).strip().lower()
    if not raw or raw=="nan":
        continue
    tri = (0, k, {"No Spot":0, "no spot":0, "very weak":1, "moderate":2, "strong":3, "very strong":4}.get(raw, row["Kd"]))
    base = matrix_parsed.at[canon, rdr]
    if tri not in base:
        base.append(tri)

# ---------- 11) OPTIONAL: Upload Luke file and integrate ----------
from google.colab import files as _files_optional

def _canon_label(s: str) -> str:
    if pd.isna(s): return ""
    return re.sub(r"\s+", "", str(s)).strip()

def _is_residue_like(s: str) -> bool:
    t = str(s).lower()
    return ("h" in t) and any(ch.isdigit() for ch in t)

def _promote_header_if_unnamed(df: pd.DataFrame) -> pd.DataFrame:
    unnamed = sum(str(c).startswith("Unnamed") for c in df.columns)
    if len(df.columns) and unnamed / len(df.columns) > 0.5 and len(df) > 1:
        new_header = df.iloc[0].astype(str).tolist()
        df = df.iloc[1:].copy()
        df.columns = new_header
    return df

def _coerce_percent_to_float(x):
    if pd.isna(x): return np.nan
    s = str(x).strip().replace(",", "")
    if not s: return np.nan
    if s.endswith("%"):
        s = s[:-1]
        try: return float(s)/100.0
        except: return np.nan
    try:
        v = float(s)
        return v if 0 <= v <= 1.2 else v/100.0
    except:
        return np.nan

def _fuzzy_sheet(xls: pd.ExcelFile, targets):
    names = list(xls.sheet_names)
    low   = [n.strip().lower() for n in names]
    tgt   = [t.strip().lower() for t in targets]
    for t in tgt:
        if t in low:
            return names[low.index(t)]
    for t in tgt:
        for i, nm in enumerate(low):
            if nm.startswith(t) or t in nm:
                return names[i]
    return None

def _detect_residue_col(df: pd.DataFrame, default_col_idx: int) -> int:
    best_j, best_hits = default_col_idx, -1
    for j, c in enumerate(df.columns):
        series = df.iloc[:, j].astype(str)
        hits = (series.map(_is_residue_like)).sum()
        if hits > best_hits:
            best_hits, best_j = hits, j
    return best_j

def _longify(df: pd.DataFrame, default_col_idx: int, sheet_tag: str):
    df = df.dropna(how="all").dropna(axis=1, how="all")
    df = _promote_header_if_unnamed(df)
    residue_col_idx  = _detect_residue_col(df, default_col_idx)
    residue_col_name = df.columns[residue_col_idx]
    reader_cols = [c for c in df.columns if c != residue_col_name]
    tidy = df[[residue_col_name] + reader_cols].copy()
    tidy.columns = ["Residue"] + [str(c).strip() for c in reader_cols]
    tidy = tidy.melt(id_vars=["Residue"], var_name="Reader", value_name=sheet_tag)
    tidy["Residue"] = tidy["Residue"].astype(str).map(_canon_label)
    tidy["Reader"]  = tidy["Reader"].astype(str).str.strip()
    return tidy

print("\nUpload Luke's wide file (optional):")
_uploaded_opt = _files_optional.upload()
luke_filename = None
if _uploaded_opt:
    luke_filename = Path(next(iter(_uploaded_opt)))

k_luke = None
luke_sheet_df_wide = None

if luke_filename:
    luke_xls = pd.ExcelFile(luke_filename)
    pe_name   = _fuzzy_sheet(luke_xls, ["PE percent positive", "pe percent positive", "pe%"])
    gate_name = _fuzzy_sheet(luke_xls, ["Accounting for Gating", "accounting for gating"])
    n_name    = _fuzzy_sheet(luke_xls, ["Number of Reader Expressing Cells", "number of reader expressing cel"])
    print("Luke sheet detection:", {"PE":pe_name, "Gate":gate_name, "N":n_name})

    if not (pe_name and gate_name and n_name):
        print("Luke: required sheets not all found; skipping Luke merge.")
    else:
        pe_df   = pd.read_excel(luke_xls, sheet_name=pe_name,   dtype=str)
        gate_df = pd.read_excel(luke_xls, sheet_name=gate_name, dtype=str)
        n_df    = pd.read_excel(luke_xls, sheet_name=n_name,    dtype=str)

        pe_long   = _longify(pe_df,   1, "pe")
        gate_long = _longify(gate_df, 0, "gate")
        n_long    = _longify(n_df,    0, "n")

        pe_long["pe"]     = pe_long["pe"].map(_coerce_percent_to_float)
        gate_long["gate"] = gate_long["gate"].map(_coerce_percent_to_float)
        def _coerce_int(x):
            if pd.isna(x): return np.nan
            try: return int(float(str(x).replace(",", "")))
            except: return np.nan
        n_long["n"] = n_long["n"].map(_coerce_int)

        lg = (pe_long
              .merge(gate_long, on=["Residue","Reader"], how="outer")
              .merge(n_long,    on=["Residue","Reader"], how="outer"))

        lg["delta"]  = (lg["pe"] - lg["gate"]).clip(lower=0)
        lg["is_hit"] = (lg["delta"] >= 0.15)
        lg_use = lg[(~lg["pe"].isna()) & (~lg["gate"].isna()) & (lg["n"] >= 1000)].copy()

        def _fmt(delta, n, hit):
            if pd.isna(delta) or pd.isna(n): return ""
            s = f"{delta:.4f} (n={int(n)})"
            return s + "*" if bool(hit) else s

        luke_wide = (lg_use
                     .assign(val=lambda d: d.apply(lambda r: _fmt(r["delta"], r["n"], r["is_hit"]), axis=1))
                     .pivot_table(index="Residue", columns="Reader", values="val", aggfunc="first")
                     .sort_index())

        luke_sheet_df_wide = luke_wide.copy()

        # Ensure Luke exists in Keys and get numeric key
        luke_source_name = "Luke's paper"
        luke_method      = "Cell-based PE−gate"
        if not (keys_df["source_name"].astype(str) == luke_source_name).any():
            next_key = (pd.to_numeric(keys_df["key"], errors="coerce").max() or 0) + 1
            keys_df = pd.concat([keys_df, pd.DataFrame([{
                "key": next_key,
                "filename": str(luke_filename.name),
                "source_name": luke_source_name,
                "URL": "",
                "measurement_method": luke_method
            }])], ignore_index=True)

        # rebuild filename→key
        ref_to_key = {}
        for _, row in keys_df.iterrows():
            for fn in str(row["filename"]).split(" , "):
                fn = fn.strip()
                if fn:
                    ref_to_key[fn] = row["key"]
        k_luke = ref_to_key.get(str(luke_filename.name))

        # Ensure readers present in matrix (for tuple storage)
        for rdr in luke_wide.columns:
            if rdr not in matrix_parsed.columns:
                matrix_parsed[rdr] = [list() for _ in range(len(matrix_parsed.index))]

        # Make sure rows exist if we later append tuples
        need_rows = [r for r in luke_wide.index
                     if _valid_residue_label(r) and r not in matrix_parsed.index]
        if need_rows:
            add = pd.DataFrame({c: [list() for _ in range(len(need_rows))] for c in matrix_parsed.columns},
                               index=need_rows)
            matrix_parsed = pd.concat([matrix_parsed, add], axis=0)

        appended = 0
for res in luke_wide.index:
    if (not _valid_residue_label(res)) or (res not in matrix_parsed.index):
        continue

for rdr in luke_wide.columns:
    if rdr not in matrix_parsed.columns:
        matrix_parsed[rdr] = [list() for _ in range(len(matrix_parsed.index))]

need_rows = [r for r in luke_wide.index
             if _valid_residue_label(r) and r not in matrix_parsed.index]
if need_rows:
    add = pd.DataFrame({c: [list() for _ in range(len(need_rows))] for c in matrix_parsed.columns},
                       index=need_rows)
    matrix_parsed = pd.concat([matrix_parsed, add], axis=0)

appended = 0

for res in luke_wide.index:
    if (not _valid_residue_label(res)) or (res not in matrix_parsed.index):
        continue
    for rdr in luke_wide.columns:
        cell_text = luke_wide.at[res, rdr]
        if not isinstance(cell_text, str) or not cell_text:
            continue

        hit = cell_text.endswith("*")
        if hit:
            cell_text = cell_text[:-1]

        m = re.match(r"^\s*([0-9]*\.?[0-9]+)\s*\(n\s*=\s*(\d+)\)\s*$", cell_text)
        if not m:
            continue

        delta_val = float(m.group(1))
        n_val     = int(m.group(2))

        base = matrix_parsed.at[res, rdr]

        if k_luke is not None and (0, k_luke) not in base:
            base.append((0, k_luke))                 # Luke presence tag

        pair = (round(delta_val, 4), int(n_val))      # Luke numeric pair
        if pair not in base:
            base.append(pair)
            appended += 1

        matrix_parsed.at[res, rdr] = base

print(f"Luke integration: rows={len(luke_wide.index)}, cols={len(luke_wide.columns)}, appended pairs={appended}")

# ---------- 12) Indices ----------
def _load_mod_index_from_sheet(xls_obj):
    try:
        df = pd.read_excel(xls_obj, "Mod Indexes", dtype={"mod": str, "index": float})
        df = df.dropna(subset=["mod", "index"]).copy()
        df["mod"] = df["mod"].astype(str).str.strip().str.lower()
        df["index"] = df["index"].astype(int)
        if df["mod"].duplicated(keep=False).any():
            d = df[df["mod"].duplicated(keep=False)].sort_values("mod")
            raise ValueError("Duplicate mod labels in 'Mod Indexes':\n" + d.to_string(index=False))
        if df["index"].duplicated(keep=False).any():
            d = df[df["index"].duplicated(keep=False)].sort_values("index")
            raise ValueError("Duplicate mod indices in 'Mod Indexes':\n" + d.to_string(index=False))
        return dict(zip(df["mod"], df["index"])), df
    except Exception as e:
        print(f"ℹ️  Using built-in mod index map (could not load/validate sheet: {e})")
        return None, None

MOD_INDEX, _mod_sheet_df_in = _load_mod_index_from_sheet(xls)
if MOD_INDEX is None:
    MOD_INDEX = {"none":0,"ac":1,"me1":2,"me2":3,"me3":4,"ph":5,"lac":6,"ub":7,"ser":8,"cit":9,"prop":10,"cro":11,"brut":12}
MOD_SYNONYM = {"me":"me1", "but":"brut"}

# =========================
# NEW (ADDITIVE): Auto-extend MOD_INDEX if new mod tags appear in residue labels
# =========================
def _discover_mod_tags_from_labels(labels):
    tags = set()
    for lbl in labels:
        if lbl is None or (isinstance(lbl, float) and np.isnan(lbl)):
            continue
        s = str(lbl).split("(")[0].strip()
        # remove whitespace
        s = re.sub(r"\s+", "", s)
        # split by "/" to catch multi-marks
        parts = s.split("/")
        for p in parts:
            # match like H3K27ac, H2BK120ub, etc.
            m = re.match(r"^H\d+(?:\.\d+)?[ABab]?[KSTRQYHDE]\d+([A-Za-z0-9]+)?$", p)
            if not m:
                continue
            raw_tag = (m.group(1) or "").lower()
            canon_tag = MOD_SYNONYM.get(raw_tag, raw_tag)
            if canon_tag == "":
                canon_tag = "none"
            tags.add(canon_tag)
    return tags

def _extend_mod_index_inplace(mod_index_map, observed_tags):
    missing = [t for t in sorted(observed_tags) if t not in mod_index_map]
    if not missing:
        return []
    max_idx = max(int(v) for v in mod_index_map.values()) if mod_index_map else 0
    added = []
    for t in missing:
        max_idx += 1
        mod_index_map[t] = max_idx
        added.append((t, max_idx))
    return added

# scan all residue labels we currently know about (matrix index + refs_df residues + Luke residues if present)
_all_res_labels = set()
_all_res_labels.update([x for x in matrix_parsed.index if _valid_residue_label(x)])
_all_res_labels.update([x for x in refs_df["Histone residues"].astype(str).tolist() if str(x).strip() != ""])
if 'luke_sheet_df_wide' in locals() and luke_sheet_df_wide is not None:
    _all_res_labels.update([x for x in luke_sheet_df_wide.index if _valid_residue_label(x)])

_observed_tags = _discover_mod_tags_from_labels(_all_res_labels)
_added_mods = _extend_mod_index_inplace(MOD_INDEX, _observed_tags)
if _added_mods:
    print("🧩 Mod Indexes updated with new tags:", ", ".join([f"{m}→{i}" for m,i in _added_mods]))

def parse_mark_full(lbl):
    s = str(lbl).split('(')[0].strip()
    mH = re.match(r'H(\d+(?:\.\d+)?)', s, flags=re.I)
    if not mH:
        return pd.Series({'histone':None,'site':[],'mod':[]})
    hist = float(mH.group(1))
    rest = s[mH.end():]
    if rest.startswith(('A','B','a','b')):
        rest = rest[1:]
    rest = rest.lstrip('/')
    sites, mods = [], []
    for p in [p for p in rest.split('/') if p]:
        m = re.match(r'([KSTRQYHDE])(\d+)([A-Za-z0-9]+)?', p.strip(), flags=re.I)
        if not m: continue
        sites.append(int(m.group(2)))
        raw_tag = (m.group(3) or "").lower()
        canon_tag = MOD_SYNONYM.get(raw_tag, raw_tag)
        if canon_tag == "":
            canon_tag = "none"
        mods.append(MOD_INDEX.get(canon_tag, 0))
    return pd.Series({'histone':hist,'site':sites,'mod':mods})

# ---------- 12b) DEDUPE numeric pairs by n (presence untouched) ----------
def _is_presence_pair2(t):
    return (isinstance(t, tuple) and len(t)==2 and t[0]==0
            and isinstance(t[1], (int,float)) and float(t[1]).is_integer())

def _dedupe_numeric_pairs(lst):
    if not isinstance(lst, list): return lst
    by_n = {}
    others = []
    for t in lst:
        if isinstance(t, tuple) and len(t)==2 and all(isinstance(x,(int,float)) for x in t):
            if _is_presence_pair2(t):
                others.append(t)
            else:
                v, n = t
                if n not in by_n or abs(v) > abs(by_n[n][0]):
                    by_n[n] = (v, n)
        else:
            others.append(t)
    return others + list(by_n.values())

matrix_parsed = matrix_parsed.applymap(_dedupe_numeric_pairs)

# ---------- 12c) Extractors + KD numeric→strength mapping ----------
_STRENGTH_WORDS = {
    "no spot":0,"nospot":0,"none":0,"0":0,"0.0":0,
    "very weak":1,"weak":1,"1":1,"1.0":1,
    "moderate":2,"mod":2,"2":2,"2.0":2,
    "strong":3,"3":3,"3.0":3,
    "very strong":4,"vstrong":4,"4":4,"4.0":4,
}
def _strength_from_any2(x):
    if isinstance(x, (int, float)):
        v = int(round(float(x)))
        return v if 0 <= v <= 4 else None
    s = str(x).strip().lower()
    if s in _STRENGTH_WORDS: return _STRENGTH_WORDS[s]
    s2 = s.replace(",", " ")
    m = re.search(r'\b([0-4])(?:\.0+)?\b', s2)
    return int(m.group(1)) if m else None

def _choose_best_source_and_strength(items):
    best = None
    for t in items:
        if isinstance(t, tuple) and len(t)>=3 and t[0]==0:
            try: key = int(t[1])
            except Exception: continue
            strength = _strength_from_any2(t[2])
            if strength is None: continue
            if best is None or strength > best[1]:
                best = (key, strength)
    return best if best else (None, None)

def _find_kd_string(items):
    def _looks_like_kd_numeric(s):
        s = str(s).strip()
        if re.fullmatch(r'[0-4](?:\.0+)?', s): return False
        return bool(re.fullmatch(r'\d*\.?\d+(?:\s*(?:nM|uM|µM|mM))?', s))
    for t in items:
        if isinstance(t, tuple) and len(t)>=3 and t[0]==0 and isinstance(t[2], str):
            s = t[2]
            if ('±' in s) or ('+/-' in s) or _looks_like_kd_numeric(s):
                try: k = int(t[1])
                except Exception: k = None
                return (k, s)
    for t in items:
        if isinstance(t, str):
            s = t
            if ('±' in s) or ('+/-' in s) or _looks_like_kd_numeric(s):
                return (None, s)
    return (None, 'NA')

def _extract_kd_uM(kd_str: str):
    if kd_str in (None, 'NA') or not isinstance(kd_str, str):
        return np.nan
    s = kd_str.strip()
    s = re.sub(r'\s*±\s*[\d\.]+','', s)
    s = re.sub(r'\s*\+/-\s*[\d\.]+','', s)
    s = re.sub(r'\([^)]*\)','', s)
    s = s.replace(',', ' ')
    m = re.search(r'([0-9]*\.?[0-9]+)\s*([munµ]M|mM|uM|nM)?', s, flags=re.I)
    if not m:
        return np.nan
    val = float(m.group(1))
    unit = m.group(2)
    if not unit:
        unit = 'uM'
    unit = unit.replace('µ','u')
    u = unit.lower()
    if u == 'nm': return val / 1000.0
    if u == 'um': return val
    if u == 'mm': return val * 1000.0
    return np.nan

def _kd_to_strength_uM(kd_uM: float):
    if not np.isfinite(kd_uM): return None
    if kd_uM <= 0.05:        return 4
    elif kd_uM <= 0.5:       return 3
    elif kd_uM <= 5:         return 2
    elif kd_uM <= 50:        return 1
    else:                    return 0

def _find_luke(items):
    for t in items:
        if isinstance(t, tuple) and len(t)==2 and all(isinstance(x,(int,float)) for x in t):
            if _is_presence_pair2(t):  # skip presence (0,key)
                continue
            v, n = t
            return (f"{v:.4f}", int(n))
    return ('NA', 'NA')

def _presence_key(items, k_luke=None):
    if k_luke is not None:
        for t in items:
            if _is_presence_pair2(t) and int(t[1]) == int(k_luke):
                return int(k_luke)
    for t in items:
        if _is_presence_pair2(t):
            return int(t[1])
    return None

# ---------- 12d) Build unified 5-field tuples ----------
# (source_key, kd_value_str_or_'NA', kd_strength_index_or_'NA', luke_delta_or_'NA', luke_n_or_'NA')
unified = pd.DataFrame(index=matrix_parsed.index, columns=matrix_parsed.columns)

k_luke_from_keys = None
try:
    k_luke_from_keys = int(keys_df.loc[keys_df['source_name']=="Luke's paper",'key'].iloc[0])
except Exception:
    k_luke_from_keys = None

for r_i in range(len(unified.index)):
    row_vals = matrix_parsed.iloc[r_i]
    for c_i in range(len(unified.columns)):
        items = row_vals.iloc[c_i]
        if not isinstance(items, list) or len(items)==0:
            unified.iat[r_i, c_i] = ""
            continue

        best_src_key, best_strength = _choose_best_source_and_strength(items)
        kd_src, kd_str              = _find_kd_string(items)
        luke_delta, luke_n          = _find_luke(items)
        present_key                 = _presence_key(items, k_luke_from_keys)

        kd_strength_from_numeric = None
        if kd_str != 'NA':
            kd_uM = _extract_kd_uM(kd_str)
            kd_strength_from_numeric = _kd_to_strength_uM(kd_uM)

        src_final = kd_src if kd_src is not None else (best_src_key if best_src_key is not None else present_key)

        strength_final = None
        if kd_strength_from_numeric is not None:
            strength_final = kd_strength_from_numeric
        elif best_strength is not None:
            strength_final = best_strength

        if any([src_final is not None, kd_str!='NA', strength_final is not None, luke_delta!='NA', luke_n!='NA']):
            unified.iat[r_i, c_i] = (
                src_final if src_final is not None else 'NA',
                kd_str,
                strength_final if strength_final is not None else 'NA',
                luke_delta, luke_n
            )
        else:
            unified.iat[r_i, c_i] = ""

# ---------- 12e) Kd_matrix_indexed with parsing columns ----------
df_kd = unified.reset_index().rename(columns={'index':'Histone residues'})
parsed_cols = df_kd['Histone residues'].apply(parse_mark_full)
df_kd[['histone','site','mod']] = parsed_cols
readers = [c for c in df_kd.columns if c not in {"Histone residues","histone","site","mod"}]

_DESC_COLS = {"Histone residues","histone","site","mod"}
_reader_cols = [c for c in df_kd.columns if c not in _DESC_COLS]

def _cell_has_payload(x):
    if isinstance(x, tuple) and len(x) == 5:
        return any(v not in ("", "NA", None) for v in x)
    if isinstance(x, str):
        return x.strip() != ""
    return pd.notna(x)

label = df_kd["Histone residues"].astype(str)
is_blank = label.map(lambda s: s.strip() == "")
is_nan_str = label.map(lambda s: s.strip().lower() == "nan")
has_any = df_kd[_reader_cols].applymap(_cell_has_payload).any(axis=1)

to_drop = (is_blank | is_nan_str) & (~has_any)
if to_drop.any():
    dropped = df_kd.loc[to_drop, "Histone residues"].astype(str).tolist()
    print(f"🧹 Removing {to_drop.sum()} unlabeled rows with no data (e.g., {', '.join(dropped[:5])})")
    df_kd = df_kd.loc[~to_drop].reset_index(drop=True)

force_drop_nan_label = False
if force_drop_nan_label:
    is_nan_str_any = df_kd["Histone residues"].astype(str).str.strip().str.lower().eq("nan")
    if is_nan_str_any.any():
        print(f"⚠️ Removing {is_nan_str_any.sum()} 'nan' residue rows by policy.")
        df_kd = df_kd.loc[~is_nan_str_any].reset_index(drop=True)

_DENYLIST_TARGETED = {
    "dnamotif","h1","h1.4","h2ax","h2aac","h2b","h2bac","h3","h3andh4",
    "h3r2","h3ac","h4","h4kac","h4ac","hkme","mcg"
}
res_norm = df_kd["Histone residues"].astype(str).map(lambda s: re.sub(r"\s+","", s).strip().lower())
is_denylisted = res_norm.isin(_DENYLIST_TARGETED)
has_any_data  = df_kd[_reader_cols].applymap(_cell_has_payload).any(axis=1)

to_drop = is_denylisted & (~has_any_data)
drop_labels = df_kd.loc[to_drop, "Histone residues"].astype(str).tolist()
if drop_labels:
    print(f"🧹 Removing {len(drop_labels)} placeholder rows with no data: "
          + ", ".join(drop_labels[:8]) + ("..." if len(drop_labels) > 8 else ""))

df_kd = df_kd.loc[~to_drop].reset_index(drop=True)

# ---------- 12f) Build "Kd_matrix_indexed_MY_ONLY" (drop Luke tuples) ----------
def _na_like(v):
    if v is None:
        return True
    if isinstance(v, float):
        try:
            return np.isnan(v)
        except Exception:
            pass
    s = str(v).strip()
    return s == "" or s.upper() == "NA"

def _is_luke_tuple_5(t):
    return isinstance(t, tuple) and len(t) == 5 and (not _na_like(t[3]) or not _na_like(t[4]))

df_kd_my_only = df_kd.copy()
for c in readers:
    df_kd_my_only[c] = df_kd_my_only[c].apply(lambda x: "" if _is_luke_tuple_5(x) else x)

# ---------- 12g) Luke's paper plus: quartiles from position 4 (ONLY), restricted to Luke's interactions ----------
# UPDATE: include ALL cells present in Luke's wide sheet (even blanks) as interactions.
# Blank/missing/≤0 pos4 → Bucket 0 ("0 - No Spot").

def _to_float_or_nan(v):
    if _na_like(v): return np.nan
    try:
        return float(str(v).strip())
    except Exception:
        return np.nan

allowed_pairs = set()
if 'luke_sheet_df_wide' in locals() and luke_sheet_df_wide is not None:
    # NEW: use full cartesian of Luke's rows × columns, regardless of value (counts blanks as interactions)
    for res in luke_sheet_df_wide.index:
        for rdr in luke_sheet_df_wide.columns:
            allowed_pairs.add((res, rdr))
else:
    # fallback: only those in df_kd that actually have Luke tuples (kept as previous behavior)
    for _, row in df_kd.iterrows():
        residue = row["Histone residues"]
        for rdr in readers:
            cell = row[rdr]
            if isinstance(cell, tuple) and len(cell) == 5 and not _na_like(cell[3]):
                allowed_pairs.add((residue, rdr))

records_pos = []
records_zero = []

for _, row in df_kd.iterrows():
    residue = row["Histone residues"]
    for rdr in readers:
        if (residue, rdr) not in allowed_pairs:
            continue  # only things present on Luke's sheet
        cell = row[rdr]
        if isinstance(cell, tuple) and len(cell) == 5:
            _, _, pos3_strength, luke_delta, _ = cell
            d = _to_float_or_nan(luke_delta)
            if not np.isfinite(d) or d <= 0.0:
                records_zero.append({
                    "Bucket": 0,
                    "Strength label": "0 - No Spot",
                    "Histone residues": residue,
                    "Reader": rdr,
                    "pos3 strength (existing)": pos3_strength if pos3_strength != "NA" else np.nan,
                    "pos4 (Luke delta)": np.nan if not np.isfinite(d) else float(d)
                })
            else:
                records_pos.append({
                    "Histone residues": residue,
                    "Reader": rdr,
                    "pos3 strength (existing)": pos3_strength if pos3_strength != "NA" else np.nan,
                    "pos4 (Luke delta)": float(d)
                })
        else:
            # Present in Luke's table but no tuple/value recorded → treat as No Spot
            records_zero.append({
                "Bucket": 0,
                "Strength label": "0 - No Spot",
                "Histone residues": residue,
                "Reader": rdr,
                "pos3 strength (existing)": np.nan,
                "pos4 (Luke delta)": np.nan
            })

if len(records_pos) == 0 and len(records_zero) == 0:
    lukes_plus_df_out = pd.DataFrame(columns=[
        "Bucket","Strength label","Histone residues","Reader","pos3 strength (existing)","pos4 (Luke delta)"
    ])
else:
    df_pos = pd.DataFrame.from_records(records_pos)
    if not df_pos.empty:
        bins = pd.qcut(df_pos["pos4 (Luke delta)"], 4, labels=[1,2,3,4], duplicates="drop")
        df_pos["Bucket"] = bins.astype(float).astype("Int64")
        label_map = {1:"1 - Very weak", 2:"2 - Moderate", 3:"3 - Strong", 4:"4 - Very Strong"}
        df_pos["Strength label"] = df_pos["Bucket"].map(label_map)
        df_pos = df_pos[[
            "Bucket","Strength label","Histone residues","Reader","pos3 strength (existing)","pos4 (Luke delta)"
        ]].sort_values(["Bucket","pos4 (Luke delta)"], ascending=[True,True])
    else:
        df_pos = pd.DataFrame(columns=[
            "Bucket","Strength label","Histone residues","Reader","pos3 strength (existing)","pos4 (Luke delta)"
        ])

    df_zero = pd.DataFrame.from_records(records_zero, columns=[
        "Bucket","Strength label","Histone residues","Reader","pos3 strength (existing)","pos4 (Luke delta)"
    ])

    lukes_plus_df_out = pd.concat([df_zero, df_pos], ignore_index=True)

# =========================
# NEW (ADDITIVE) 12h) Luke's paper plus HITS:
#   - delta < 0.15  -> Bucket 0 ("0 - No Spot")
#   - delta >= 0.15 -> quartiles into 1..4, based ONLY on those >= 0.15
#   - still restricted to Luke’s interaction space (allowed_pairs)
# =========================
HIT_THRESHOLD = 0.15

records_hit_pos = []
records_hit_zero = []

for _, row in df_kd.iterrows():
    residue = row["Histone residues"]
    for rdr in readers:
        if (residue, rdr) not in allowed_pairs:
            continue
        cell = row[rdr]
        if isinstance(cell, tuple) and len(cell) == 5:
            _, _, pos3_strength, luke_delta, _ = cell
            d = _to_float_or_nan(luke_delta)

            # Anything blank/NA or below threshold => 0
            if (not np.isfinite(d)) or (d < HIT_THRESHOLD):
                records_hit_zero.append({
                    "Bucket": 0,
                    "Strength label": "0 - No Spot",
                    "Histone residues": residue,
                    "Reader": rdr,
                    "pos3 strength (existing)": pos3_strength if pos3_strength != "NA" else np.nan,
                    "pos4 (Luke delta)": np.nan if not np.isfinite(d) else float(d)
                })
            else:
                records_hit_pos.append({
                    "Histone residues": residue,
                    "Reader": rdr,
                    "pos3 strength (existing)": pos3_strength if pos3_strength != "NA" else np.nan,
                    "pos4 (Luke delta)": float(d)
                })
        else:
            # Present in Luke's table but no tuple/value recorded -> treat as below threshold => 0
            records_hit_zero.append({
                "Bucket": 0,
                "Strength label": "0 - No Spot",
                "Histone residues": residue,
                "Reader": rdr,
                "pos3 strength (existing)": np.nan,
                "pos4 (Luke delta)": np.nan
            })

if len(records_hit_pos) == 0 and len(records_hit_zero) == 0:
    lukes_plus_hits_df_out = pd.DataFrame(columns=[
        "Bucket","Strength label","Histone residues","Reader","pos3 strength (existing)","pos4 (Luke delta)"
    ])
else:
    df_hit_pos = pd.DataFrame.from_records(records_hit_pos)
    if not df_hit_pos.empty:
        bins = pd.qcut(df_hit_pos["pos4 (Luke delta)"], 4, labels=[1,2,3,4], duplicates="drop")
        df_hit_pos["Bucket"] = bins.astype(float).astype("Int64")
        label_map = {1:"1 - Very weak", 2:"2 - Moderate", 3:"3 - Strong", 4:"4 - Very Strong"}
        df_hit_pos["Strength label"] = df_hit_pos["Bucket"].map(label_map)
        df_hit_pos = df_hit_pos[[
            "Bucket","Strength label","Histone residues","Reader","pos3 strength (existing)","pos4 (Luke delta)"
        ]].sort_values(["Bucket","pos4 (Luke delta)"], ascending=[True,True])
    else:
        df_hit_pos = pd.DataFrame(columns=[
            "Bucket","Strength label","Histone residues","Reader","pos3 strength (existing)","pos4 (Luke delta)"
        ])

    df_hit_zero = pd.DataFrame.from_records(records_hit_zero, columns=[
        "Bucket","Strength label","Histone residues","Reader","pos3 strength (existing)","pos4 (Luke delta)"
    ])

    lukes_plus_hits_df_out = pd.concat([df_hit_zero, df_hit_pos], ignore_index=True)

# =========================
# NEW (ADDITIVE) 12i) Propagate Luke HIT-buckets into Kd_matrix_indexed (pos3) for the same interactions
# Rule: Only fill pos3 if it's currently NA/blank (do NOT overwrite existing pos3 strength).
# =========================
_bucket_map_hits = {}
try:
    for _, rr in lukes_plus_hits_df_out.iterrows():
        _bucket_map_hits[(str(rr["Histone residues"]), str(rr["Reader"]))] = int(rr["Bucket"]) if pd.notna(rr["Bucket"]) else 0
except Exception:
    _bucket_map_hits = {}

def _fill_pos3_from_luke_hits(cell, residue, reader):
    if not (isinstance(cell, tuple) and len(cell) == 5):
        return cell
    src, kd_str, pos3, luke_delta, luke_n = cell
    # only apply when interaction exists in Luke space
    if (str(residue), str(reader)) not in _bucket_map_hits:
        return cell
    # only fill if pos3 is NA/blank
    if _na_like(pos3):
        b = _bucket_map_hits[(str(residue), str(reader))]
        return (src, kd_str, b, luke_delta, luke_n)
    return cell

# apply fill across df_kd (but only reader columns)
_res_list = df_kd["Histone residues"].astype(str).tolist()
for i, residue in enumerate(_res_list):
    for rdr in readers:
        df_kd.at[i, rdr] = _fill_pos3_from_luke_hits(df_kd.at[i, rdr], residue, rdr)

# ---------- 13) Index tabs ----------
# Mod Indexes: now rebuild from (possibly-extended) MOD_INDEX to ensure it updates when new mods appear
mod_index_df = pd.DataFrame(
    [{"mod":k, "index":v} for k, v in sorted(MOD_INDEX.items(), key=lambda kv: kv[1])],
    columns=["mod","index"]
)

hist_series = pd.to_numeric(df_kd['histone'], errors='coerce')
hist_vals = sorted([v for v in hist_series.dropna().unique()])
if 2.0 in hist_vals:
    hist_index_df = pd.DataFrame(
        [{'histone':'H2A','index':2},{'histone':'H2B','index':2}] +
        [{'histone':int(h) if float(h).is_integer() else h,'index':h} for h in hist_vals if h!=2.0]
    )
else:
    hist_index_df = pd.DataFrame([{'histone':h,'index':h} for h in hist_vals])

site_vals = set()
for sub in df_kd['site']:
    if isinstance(sub, list):
        site_vals.update(sub)
site_index_df = pd.DataFrame([{'site':s,'index':s} for s in sorted(site_vals)])

ik = []
for i, r in enumerate(readers, start=1):
    ik.append({'type':'reader','label':r,'index':i})
for i, res in enumerate(df_kd['Histone residues'], start=1):
    ik.append({'type':'residue','label':res,'index':i})
index_keys_df = pd.DataFrame(ik, columns=['type','label','index'])

# ---------- 13b) Source 11 sheet (legacy 11 only) ----------
def _only_source_key_triples(lst, key):
    if not isinstance(lst, list): return []
    return [t for t in lst if isinstance(t, tuple) and len(t)>=3 and t[1]==key]
s11_only = matrix_parsed.applymap(lambda lst: _only_source_key_triples(lst, 11))
s11_df_wide = s11_only.map(list_to_tuple_cell).reset_index().rename(columns={'index':'Histone residues'})
s11_parsed = s11_df_wide['Histone residues'].apply(parse_mark_full)
s11_df_wide[['histone','site','mod']] = s11_parsed
s11_readers = [c for c in s11_df_wide.columns if c not in {"Histone residues","histone","site","mod"}]
s11_df_out = s11_df_wide[['histone','site','mod','Histone residues'] + s11_readers]
s11_title_safe = "Source 11"

# =========================
# NEW (ADDITIVE) 13c) Re-add Sequences sheet (preserve existing sequences if present)
# =========================
_GENERAL_HISTONES = ["H1.4", "H2", "H1B", "H2A", "H2B", "H3", "H4"]

def _normalize_sequences_sheet(df):
    # accept flexible headers; normalize to Type/Name/Sequence
    col_map = {}
    for c in df.columns:
        lc = str(c).strip().lower()
        if lc in {"type", "category"}:
            col_map[c] = "Type"
        elif lc in {"name", "protein", "histone", "reader"}:
            col_map[c] = "Name"
        elif "seq" in lc:
            col_map[c] = "Sequence"
    df2 = df.rename(columns=col_map).copy()
    for need in ["Type", "Name", "Sequence"]:
        if need not in df2.columns:
            df2[need] = ""
    df2["Type"] = df2["Type"].astype(str).str.strip()
    df2["Name"] = df2["Name"].astype(str).str.strip()
    df2["Sequence"] = df2["Sequence"].astype(str)
    df2 = df2[["Type", "Name", "Sequence"]]
    # drop fully blank names
    df2 = df2[df2["Name"].astype(str).str.strip().ne("")]
    return df2

_existing_sequences_df = None
if "Sequences" in xls.sheet_names:
    try:
        _existing_sequences_df = pd.read_excel(xls, "Sequences", dtype=str)
        _existing_sequences_df = _normalize_sequences_sheet(_existing_sequences_df)
    except Exception as e:
        print(f"⚠️ Could not read existing 'Sequences' sheet; will recreate it. Reason: {e}")
        _existing_sequences_df = None

# build base entries
_seq_rows = []
for h in _GENERAL_HISTONES:
    _seq_rows.append({"Type": "Histone", "Name": h, "Sequence": ""})
for r in readers:
    _seq_rows.append({"Type": "Reader", "Name": str(r).strip(), "Sequence": ""})

_sequences_df = pd.DataFrame(_seq_rows)

# merge-preserve: keep any existing sequences, only add missing rows
if _existing_sequences_df is not None and len(_existing_sequences_df) > 0:
    # outer merge on Type+Name, prefer existing Sequence when non-empty
    merged = _sequences_df.merge(
        _existing_sequences_df,
        on=["Type", "Name"],
        how="left",
        suffixes=("", "_old")
    )
    def _pick_seq(row):
        old = row.get("Sequence_old", "")
        new = row.get("Sequence", "")
        old_s = "" if pd.isna(old) else str(old)
        new_s = "" if pd.isna(new) else str(new)
        return old_s if old_s.strip() != "" else new_s
    merged["Sequence"] = merged.apply(_pick_seq, axis=1)
    _sequences_df = merged[["Type", "Name", "Sequence"]]

# sort nicely
_type_order = {"Histone": 0, "Reader": 1}
_sequences_df["_t"] = _sequences_df["Type"].map(lambda x: _type_order.get(str(x), 99))
_sequences_df = _sequences_df.sort_values(["_t", "Name"]).drop(columns=["_t"]).reset_index(drop=True)

# ---------- 14) Write back ----------
with pd.ExcelWriter(kd_file, engine='openpyxl') as w:
    df_kd.set_index("Histone residues").to_excel(w, "Kd_matrix_indexed")
    df_kd_my_only.set_index("Histone residues").to_excel(w, "Kd_matrix_indexed_MY_ONLY")
    HEATMAP_REF = "heatmap_matrix_output_cleaned_corrected.xlsx"
    mask_heatmap = refs_df["Reference"].astype(str).str.strip().eq(HEATMAP_REF)
    refs_df.loc[mask_heatmap, "kd_idx_auto"] = np.nan
    refs_df.to_excel(w, "References", index=False)
    keys_df.to_excel(w, "Keys", index=False)
    index_keys_df.to_excel(w, "Index Keys", index=False)
    mod_index_df.to_excel(w, "Mod Indexes", index=False)
    hist_index_df.to_excel(w, "Histone number indexes", index=False)
    site_index_df.to_excel(w, "Histone site indexes", index=False)
    pd.DataFrame({
        "Kd Indicator":["No Spot","Very weak","Moderate","Strong","Very Strong"],
        "Index":[0,1,2,3,4]
    }).to_excel(w, "Kd Index Keys", index=False)
    s11_df_out.set_index("Histone residues").to_excel(w, "Source 11")
    if 'luke_sheet_df_wide' in locals() and luke_sheet_df_wide is not None:
        luke_core = (luke_sheet_df_wide.reset_index()
                     .rename(columns={'index':'Histone residues', 'Residue':'Histone residues'}))
        luke_core[['histone','site','mod']] = luke_core['Histone residues'].apply(parse_mark_full)
        (luke_core[['histone','site','mod','Histone residues'] + list(luke_sheet_df_wide.columns)]
         .set_index("Histone residues")
         .to_excel(w, "Luke's paper"))
    if 'lukes_plus_df_out' in locals() and lukes_plus_df_out is not None:
        lukes_plus_df_out.to_excel(w, "Luke's paper plus", index=False)

    # NEW (ADDITIVE): write the hit-thresholded version right after
    if 'lukes_plus_hits_df_out' in locals() and lukes_plus_hits_df_out is not None:
        lukes_plus_hits_df_out.to_excel(w, "Luke's paper plus hits", index=False)

    # NEW (ADDITIVE): write Sequences sheet back
    _sequences_df.to_excel(w, "Sequences", index=False)

# --- Auto-download the updated workbook (Colab only) ---
try:
    from google.colab import files as _colab_files
    _colab_files.download(str(kd_file))
    print(f"📥 Auto-download started for: {kd_file.name}")
except Exception as e:
    print(f"⚠️ Auto-download failed: {e}")

# ---------- 15) Interactive lookup ----------
def _build_lookup_maps(index_keys_df, df_kd, readers):
    rdr_map = {int(row['index']): row['label'] for _, row in index_keys_df[index_keys_df['type']=='reader'].iterrows()}
    res_map = {int(row['index']): row['label'] for _, row in index_keys_df[index_keys_df['type']=='residue'].iterrows()}
    rdr_norm = {re.sub(r"\s+","",c).lower(): c for c in readers}
    res_norm = {re.sub(r"\s+","",r).lower(): r for r in df_kd["Histone residues"].tolist()}
    return rdr_map, res_map, rdr_norm, res_norm

def _pick_one(options, header="Multiple matches; choose one"):
    for i, s in enumerate(options, 1):
        print(f"   {i}) {s}")
    sel = input(f"{header} (1-{len(options)}, blank to cancel): ").strip()
    if sel.isdigit() and 1 <= int(sel) <= len(options):
        return options[int(sel)-1]
    return None

def _fuzzy_match_name(name, pool):
    q = re.sub(r"\s+","", str(name)).lower()
    if q in pool: return pool[q]
    hits = [pool[k] for k in pool.keys() if q in k]
    return hits

def _lookup_ui(df_kd, readers, index_keys_df):
    rdr_map, res_map, rdr_norm, res_norm = _build_lookup_maps(index_keys_df, df_kd, readers)
    def _get_tuple(res_label, rdr_label):
        if res_label not in df_kd["Histone residues"].values:
            return None, f"Residue '{res_label}' not found."
        if rdr_label not in readers:
            return None, f"Reader '{rdr_label}' not found."
        row = df_kd.loc[df_kd["Histone residues"]==res_label].iloc[0]
        return row[rdr_label], None
    print("\n🔎 Interactive lookup:")
    print("   (n) name pair   (i) index pair   (s) search names   (q) quit")
    while True:
        mode = input("Lookup mode [n/i/s/q]: ").strip().lower()
        if mode in {"q",""}: break
        if mode == "s":
            q = input(" Search term (reader or residue): ").strip()
            r_hits = [v for k,v in rdr_norm.items() if q.lower() in k]
            e_hits = [v for k,v in res_norm.items() if q.lower() in k]
            print(f" Readers ({len(r_hits)}): " + (", ".join(r_hits[:12]) + ("..." if len(r_hits)>12 else "")))
            print(f" Residues ({len(e_hits)}): " + (", ".join(e_hits[:12]) + ("..." if len(e_hits)>12 else "")))
            continue
        if mode == "i":
            try:
                ri = int(input(" Reader index (from 'Index Keys'): ").strip())
                ei = int(input(" Residue index (from 'Index Keys'): ").strip())
            except:
                print("  ❌ Please enter integers."); continue
            if ri not in rdr_map or ei not in res_map:
                print("  ❌ Index not found in 'Index Keys'."); continue
            rdr_label = rdr_map[ri]; res_label = res_map[ei]
            val, err = _get_tuple(res_label, rdr_label)
            if err: print("  ❌", err)
            else:   print(f"  → {rdr_label} × {res_label} = {val if val!='' else '(empty)'}")
            continue
        if mode == "n":
            r_in = input(" Reader name (full or part): ").strip()
            e_in = input(" Residue name (full or part): ").strip()
            r_match = _fuzzy_match_name(r_in, rdr_norm)
            rdr_label = _pick_one(r_match, "Pick reader") if isinstance(r_match, list) else r_match
            if not rdr_label: print("  ❌ No reader match."); continue
            e_match = _fuzzy_match_name(e_in, res_norm)
            res_label = _pick_one(e_match, "Pick residue") if isinstance(e_match, list) else e_match
            if not res_label: print("  ❌ No residue match."); continue
            val, err = _get_tuple(res_label, rdr_label)
            if err: print("  ❌", err)
            else:   print(f"  → {rdr_label} × {res_label} = {val if val!='' else '(empty)'}")

try:
    _lookup_ui(df_kd, readers, index_keys_df)
except Exception as e:
    print("Interactive lookup skipped due to error:", e)

print("\n✅ Done. Sheets written:")
print(" - 'Luke's paper plus' (quartiles based on pos4 > 0.0; blanks/≤0 => 0)")
print(f" - 'Luke's paper plus hits' (thresholded: pos4 < {HIT_THRESHOLD} => 0; pos4 ≥ {HIT_THRESHOLD} quartiled into 1–4)")
print(" - 'Kd_matrix_indexed' now fills pos3 with Luke hit-buckets ONLY when pos3 was NA/blank (no overwrites).")
print(" - 'Mod Indexes', 'Histone number indexes', 'Histone site indexes' are rebuilt to reflect newly observed data.")
print(" - 'Sequences' restored (preserves existing sequences if present; adds missing histones/readers).")


🔄 Upload your latest kd_master .xlsx:


Saving kd_master_new_updated (2) (1) (7) (1) (2) (1) (5) (1) (1) (1) (1) (1) (1) (1) (1) (4) (4) (1) (1) (1) (1) (1) (1) (2) (6) (3) (1).xlsx to kd_master_new_updated (2) (1) (7) (1) (2) (1) (5) (1) (1) (1) (1) (1) (1) (1) (1) (4) (4) (1) (1) (1) (1) (1) (1) (2) (6) (3) (1).xlsx

Enter NEW overlay files (CSV/XLSX); blank to stop.
 Already have: genes_clean_expanded.csv, EpigenPandas.xlsx, https://pmc.ncbi.nlm.nih.gov/articles/instance/3326523/bin/mmc4.pdf, table_s6_full.xlsx, heatmap_matrix_output_cleaned_corrected.xlsx, KdAdditions.xlsx, Structural Basis for Acetylated Histone H4 Recognition by the Human BRD2 Broodomain.xlsx, Specificity of the HP1 chromo domain for the methylated N‐terminus of histone H3.xlsx, Selective recognition of methylated lysine 9 on histone H3 by the HP1 chromo domain.xlsx, Progress in the Discovery of Small-Molecule Inhibitors of Bromodomain–Histone Interactions.xlsx, Improved methods for the detection of histone interactions with peptide microarrays.xlsx, H

/tmp/ipython-input-1565898765.py:202: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  matrix_parsed = matrix.applymap(parse_tuple_cell_to_list)



(Optional) Upload a BASELINE workbook (has the original (0, 11, strength) triples) to rescue Source 11:


Saving kd_master_new_updated (2) (1) (7) (1) (2) (1) (5) (1) (1).xlsx to kd_master_new_updated (2) (1) (7) (1) (2) (1) (5) (1) (1).xlsx


...

🧹 Removing 16 placeholder rows with no data: DNA motif, H1, H1.4, H2AX, H2Aac, H2B, H2Bac, H3...


/tmp/ipython-input-1565898765.py:1161: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  s11_only = matrix_parsed.applymap(lambda lst: _only_source_key_triples(lst, 11))
/tmp/ipython-input-1565898765.py:1240: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  df_kd.set_index("Histone residues").to_excel(w, "Kd_matrix_indexed")
/tmp/ipython-input-1565898765.py:1241: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  df_kd_my_only.set_index("Histone residues").to_excel(w, "Kd_matrix_indexed_MY_ONLY")
/tmp/ipython-input-1565898765.py:1245: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  refs_df.to_excel(w, "References", index=False)
/tmp/ipython-input-1565898765.py:1246: FutureWarning: Starting with

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

📥 Auto-download started for: kd_master_new_updated (2) (1) (7) (1) (2) (1) (5) (1) (1) (1) (1) (1) (1) (1) (1) (4) (4) (1) (1) (1) (1) (1) (1) (2) (6) (3) (1).xlsx

🔎 Interactive lookup:
   (n) name pair   (i) index pair   (s) search names   (q) quit
Lookup mode [n/i/s/q]: q

✅ Done. Sheets written:
 - 'Luke's paper plus' (quartiles based on pos4 > 0.0; blanks/≤0 => 0)
 - 'Luke's paper plus hits' (thresholded: pos4 < 0.15 => 0; pos4 ≥ 0.15 quartiled into 1–4)
 - 'Kd_matrix_indexed' now fills pos3 with Luke hit-buckets ONLY when pos3 was NA/blank (no overwrites).
 - 'Mod Indexes', 'Histone number indexes', 'Histone site indexes' are rebuilt to reflect newly observed data.
 - 'Sequences' restored (preserves existing sequences if present; adds missing histones/readers).
